In [9]:
# new vers

import requests
import pandas as pd
# from pandasgui import show
from alpha_vantage.alphavantage import AlphaVantage
from alpha_vantage.timeseries import TimeSeries

def display_all(df):
    with pd.option_context("display.max_rows", 100000, "display.max_columns", 100000): 
        display(df)

# ///////////////////////////////////////////////////////////
# FUNCTIONS FOR ALPHA VANTAGE
# ///////////////////////////////////////////////////////////
TIME_SERIES_INTRADAY = 'TIME_SERIES_INTRADAY'
TIME_SERIES_INTRADAY_EXTENDED = 'TIME_SERIES_INTRADAY_EXTENDED'
TIME_SERIES_DAILY = 'TIME_SERIES_DAILY'
TIME_SERIES_DAILY_ADJUSTED = 'TIME_SERIES_DAILY_ADJUSTED'
TIME_SERIES_WEEKLY = 'TIME_SERIES_WEEKLY'
TIME_SERIES_WEEKLY_ADJUSTED = 'TIME_SERIES_WEEKLY_ADJUSTED'
TIME_SERIES_MONTHLY = 'TIME_SERIES_MONTHLY'
TIME_SERIES_MONTHLY_ADJUSTED = 'TIME_SERIES_MONTHLY_ADJUSTED'
GLOBAL_QUOTE = 'GLOBAL_QUOTE'
SYMBOL_SEARCH = 'SYMBOL_SEARCH'
OVERVIEW = 'OVERVIEW'
EARNINGS = 'EARNINGS'
INCOME_STATEMENT = 'INCOME_STATEMENT'
BALANCE_SHEET = 'BALANCE_SHEET'
CASH_FLOW = 'CASH_FLOW'
LISTING_STATUS = 'LISTING_STATUS'
EARNINGS_CALENDAR = 'EARNINGS_CALENDAR'
IPO_CALENDAR = 'IPO_CALENDAR'
CURRENCY_EXCHANGE_RATE = 'CURRENCY_EXCHANGE_RATE'
FX_INTRADAY = 'FX_INTRADAY'
FX_DAILY = 'FX_DAILY'
FX_WEEKLY = 'FX_WEEKLY'
FX_MONTHLY = 'FX_MONTHLY'
CURRENCY_EXCHANGE_RATE = 'CURRENCY_EXCHANGE_RATE'
CRYPTO_RATING = 'CRYPTO_RATING'
CRYPTO_INTRADAY = 'CRYPTO_INTRADAY'
DIGITAL_CURRENCY_DAILY = 'DIGITAL_CURRENCY_DAILY'
DIGITAL_CURRENCY_WEEKLY = 'DIGITAL_CURRENCY_WEEKLY'
DIGITAL_CURRENCY_MONTHLY = 'DIGITAL_CURRENCY_MONTHLY'

# ///////////////////////////////////////////////////////////
# TIME SERIES FOR STOCK DATA
# ///////////////////////////////////////////////////////////
ONEMIN = 'Time Series (1min)'
FIVEMIN = 'Time Series (5min)'
FIVETEENMIN = 'Time Series (15min)'
THIRTYMIN = 'Time Series (30min)'
HOUR = 'Time Series (60min)'
FIVEMIN = 'Time Series (5min)'
DAILY = 'Time Series (Daily)' 
WEEKLY = 'Weekly Time Series'
WEEKLY_ADJ = 'Weekly Adjusted Time Series'
MONTLY = 'Monthly Time Series'
MONTLY_ADJ = 'Monthly Adjusted Time Series'
# /////////////////////////////////////////////////////////
# query fundamental stock data of selected type ///////////
# /////////////////////////////////////////////////////////
def query_fundamental_data(func, symbol, apikey, outputsize='full', datatype='json'):
    data = {
        "function": func,
        "symbol": symbol,
        "outputsize": outputsize, # (full) and (compact) are accepted
        "datatype": datatype,
        "apikey": apikey
    }
    return requests.get("https://www.alphavantage.co/query", data).json()

def get_fundamental_dataframe(json):
    df = pd.DataFrame(json['annualReports'])
    df.set_index('fiscalDateEnding', inplace=True)
    return df

# FOR EARNINGS
def get_fundamental_dataframe1(json):
    df = pd.DataFrame(json['annualEarnings'])
    df.set_index('fiscalDateEnding', inplace=True)
    return df

#For Overview
def get_fundamental_dataframe2(json):
    df = pd.DataFrame(json, index=[1])
#     df.set_index(inplace=True)
    return df

def query_timeseries_data(func, symbol, apikey, outputsize='full', datatype='json'):
    data = {
        "function": func,
        "symbol": symbol,
        "outputsize": outputsize, # (full) and (compact) are accepted
        "datatype": datatype,
        "apikey": apikey
    }
    return requests.get("https://www.alphavantage.co/query", data).json()

def get_timeseries_dataframe(json, type):
    df = pd.DataFrame.from_dict(json[type], orient= 'index')
    df.index =  pd.to_datetime(df.index, format='%Y-%m-%d')
    
    df = df.rename(columns={ '1. open': 'Open', '2. high': 'High', '3. low': 'Low', '4. close': 'Close', '5. volume': 'Volume'})
    df = df.astype({'Open': 'float64', 'High': 'float64', 'Low': 'float64','Close': 'float64','Volume': 'float64',})
    df = df[[ 'Open', 'High', 'Low', 'Close', 'Volume']]
    return df

def get_data(stock, apikey):
    import time

    response_json = query_fundamental_data(INCOME_STATEMENT, stock, apikey)
    INCOME_DATA = get_fundamental_dataframe(response_json)
    time.sleep(12)

    response_json = query_fundamental_data(BALANCE_SHEET, stock, apikey)
    BALANCE_DATA = get_fundamental_dataframe(response_json)
    time.sleep(12)

    response_json = query_fundamental_data(CASH_FLOW, stock, apikey)
    CASHFLOW_DATA = get_fundamental_dataframe(response_json)
    time.sleep(12)

    response_json = query_fundamental_data(EARNINGS, stock, apikey)
    EARNINGS_DATA = get_fundamental_dataframe1(response_json)
    time.sleep(12)

    response_json = query_fundamental_data(OVERVIEW, stock, apikey)
    OVERVIEW_DATA = get_fundamental_dataframe2(response_json)
    time.sleep(12)

    response_json = query_timeseries_data(TIME_SERIES_DAILY, stock, apikey)
    STOCKPRICE_DATA = get_timeseries_dataframe(response_json, DAILY)
    time.sleep(12)
    
    return INCOME_DATA,BALANCE_DATA,CASHFLOW_DATA,OVERVIEW_DATA,EARNINGS_DATA,STOCKPRICE_DATA
# response_json1,response_json2,response_json3,response_json4
# ,INCOME_DATA,BALANCE_DATA,CASHFLOW_DATA,EARNINGS_DATA,OVERVIEW_DATA


In [10]:
# TESTING 

stocks = ['AAPL']
API = ['53K56OJAX5T2DQJZ']
for x,y in zip(stocks,API):
    print(x,y)
    INCOME_DATA,BALANCE_DATA,CASHFLOW_DATA,OVERVIEW_DATA,EARNINGS_DATA,STOCKPRICE_DATA = get_data(x,y)

    BALANCE_DATA = BALANCE_DATA.drop(['reportedCurrency'], axis = 1)
    CASHFLOW_DATA = CASHFLOW_DATA.drop(['reportedCurrency','netIncome'], axis = 1)

    # Define percentage change formula for EARNING data
    EARNINGS_DATA = EARNINGS_DATA.apply(pd.to_numeric)
    EARNINGS_DATA['shifted_EPS'] = EARNINGS_DATA.reportedEPS.shift(-1, axis = 0)
    EARNINGS_DATA['EPS_Growth_Rate']= 100*((EARNINGS_DATA['reportedEPS'] - EARNINGS_DATA['shifted_EPS'])/ abs(EARNINGS_DATA['shifted_EPS']))

    EARNINGS_DATA[['EPS_Growth_Avg_Perc']] = EARNINGS_DATA[:6]['EPS_Growth_Rate'].mean()

    # EARNINGS_DATA1 = EARNINGS_DATA1.reset_index(0).reset_index(drop=True)
    # EARNINGS_DATA1
    IBC = pd.merge(pd.merge(pd.merge(INCOME_DATA,BALANCE_DATA,on='fiscalDateEnding')
                            ,CASHFLOW_DATA,on='fiscalDateEnding')
                   ,EARNINGS_DATA,on='fiscalDateEnding')
    IBC = IBC.reset_index(0).reset_index(drop=True)

    IBC.replace('None', value=0, inplace=True)

    # # To replace change type to int after the column of 'reportedcurrency'
    index_no = IBC.columns.get_loc('reportedCurrency')
    index_no_starting = index_no+1
    IBC.iloc[:,index_no_starting:] = IBC.iloc[:,index_no_starting:].apply(pd.to_numeric)


    IBC['TotalDebt_FOR_DERatio'] = IBC['shortTermDebt']+IBC['longTermDebt']
    IBC['D/E Ratio'] = IBC['TotalDebt_FOR_DERatio']/IBC['totalShareholderEquity']

    IBC1 = IBC.copy()
    IBC1= pd.concat([OVERVIEW_DATA.reset_index(drop=True),IBC1.iloc[:1,:]], axis=1)
    IBC1['EntryPrice'] = IBC1['EPS_Growth_Avg_Perc'] * float(IBC1['EPS'])



    price = STOCKPRICE_DATA.iloc[:1,:].reset_index(drop=True)    
    IBC1 = pd.concat([IBC1,price], axis=1)

    IBC1['new_EntryPrice'] = [-x if y <0 else x for x,y in zip(IBC1['EntryPrice'],IBC1['EPS_Growth_Avg_Perc'])]

AAPL 53K56OJAX5T2DQJZ


In [11]:
IBC1

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EPS_Growth_Avg_Perc,TotalDebt_FOR_DERatio,D/E Ratio,EntryPrice,Open,High,Low,Close,Volume,new_EntryPrice
0,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,USA,TECHNOLOGY,ELECTRONIC COMPUTERS,...,3.718265,140913000000,2.233524,20.859465,169.08,172.5,168.17,172.19,106765552.0,20.859465


In [12]:
# Final 
def extractdata(stock,EarningorRevenue):
    from bs4 import BeautifulSoup
    
    # EarningorRevenue = [0] => Earning Estimate
    # EarningorRevenue = [1] => Revenue Estimate
    headers = {'User-Agent': 'Mozilla/5.0'}

    url = f"https://finance.yahoo.com/quote/{stock}/analysis?p={stock}"

    r = requests.get(url, headers=headers)
    # print(r.text)

    soup = BeautifulSoup(r.text, 'html.parser')
    # print(soup.prettify())
    Estimate_Table = soup.find_all("table", attrs={"class": "W(100%) M(0) BdB Bdc($seperatorColor) Mb(25px)"})
    table_thead = Estimate_Table[EarningorRevenue].thead.find_all("th")
    table_body = Estimate_Table[EarningorRevenue].find("tbody")

    headingitem = []
    table_data = []
    for i in table_thead:
    #     print(i.text)
        headingitem.append(i.text)
    for body in table_body:
        row_list = []
        for row in body:
            for item in row:
                row_list.append(item.text)        
        table_data.append(row_list)
    df = pd.DataFrame(table_data, columns = [headingitem])
    return df

In [13]:
Revenue_Estimate = extractdata('AAPL',1)
Revenue_Estimate

,Revenue Estimate,Current Qtr. (Dec 2021),Next Qtr. (Mar 2022),Current Year (2022),Next Year (2023)
0,No. of Analysts,22,22,37,32
1,Avg. Estimate,118.17B,90.41B,381.75B,401.25B
2,Low Estimate,111.81B,83.26B,359.16B,300.5B
3,High Estimate,121.3B,98.28B,405.44B,445.58B
4,Year Ago Sales,N/A,N/A,365.82B,381.75B
5,Sales Growth (year/est),N/A,N/A,4.40%,5.10%


In [ ]:
def changeMoneytoNumbers(df,targetRow,colnum = 3):
    m = {'K': 3, 'M': 6, 'B': 9, 'T': 12}
    col = Revenue_Estimate.columns.to_list()[colnum]
    firstcol = df.columns.to_list()[0]
    df = df[df[firstcol] == targetRow]
    df[col] = [int(float(i[:-1]) * 10 ** m[i[-1]]) for i in df[col]]
    return(df)


targetRow = 'Avg. Estimate'
df = Revenue_Estimate
df1 = changeMoneytoNumbers(df,targetRow,colnum = 3)
df1 = changeMoneytoNumbers(df1,targetRow,colnum = 4)
# df1

#  to get analyst value
analystval = df1.iloc[:,-2:].values.tolist()[0]

analystval[0], analystval[1] = analystval[1], analystval[0]
analystval

In [ ]:
IBC_DCF = IBC.copy()
IBC_DCF

In [ ]:
col = ['fiscalDateEnding','totalRevenue','netIncome','operatingCashflow','capitalExpenditures']
IBC_DCF1 = IBC_DCF[col]
IBC_DCF1['FreeCashFlowToEquity'] = IBC_DCF1['operatingCashflow'] - IBC_DCF1['capitalExpenditures']
IBC_DCF1['FCFE/NetIncome(100%)'] = (IBC_DCF1['FreeCashFlowToEquity'] / IBC_DCF1['netIncome'])*100
IBC_DCF1['Lowest_FCFE/NetIncome(100%)'] = IBC_DCF1['FCFE/NetIncome(100%)'].min()

#create 2 years of blank data
listofna = (len(IBC_DCF1.columns)-1)*[0]
date = ['2025-01-22','2024-01-22','2023-01-22','2022-01-22']
row=[]
for i in date:
    onedate = []
    onedate.append(i)
    for j in listofna:
        onedate.append(j)
    row.append(onedate)
    
# putting blank row into the df    
colss = IBC_DCF1.columns.to_list()
newrow = pd.DataFrame(row,columns = colss).reset_index(drop=True)
IBC_DCF1 = pd.concat([newrow, IBC_DCF1], ignore_index=True, axis=0)

# Replace the analyst avg rating into the df
IBC_DCF1.iloc[2:4,1] = analystval


IBC_DCF1

In [ ]:

def totalrevenueExtension(df,rownum):
    df['totalRevenue'].iloc[rownum] = ( df['totalRevenue'].iloc[rownum+1]*(100 +  df['totalRevenue_Growth_Rate_Mean'].iloc[rownum+1]))/100
    return df

In [ ]:
IBC_DCF2 = IBC_DCF1.copy()
IBC_DCF2['shifted_revenue'] = IBC_DCF2.totalRevenue.shift(-1, axis = 0)
IBC_DCF2['totalRevenue_Growth_Rate(100%)']= 100*((IBC_DCF2['totalRevenue'] - IBC_DCF2['shifted_revenue'])/ abs(IBC_DCF2['shifted_revenue']))
IBC_DCF2['totalRevenue_Growth_Rate(100%)'][1] = 0
IBC_DCF2['totalRevenue_Growth_Rate_Mean'] = IBC_DCF2['totalRevenue_Growth_Rate(100%)'].mean()
# IBC_DCF2['totalRevenue'].iloc[1] = ( IBC_DCF2['totalRevenue'].iloc[2]*(100 +  IBC_DCF2['totalRevenue_Growth_Rate_Mean'].iloc[2]))/100
IBC_DCF2 = totalrevenueExtension(IBC_DCF2,1)
IBC_DCF2 = totalrevenueExtension(IBC_DCF2,0)
IBC_DCF2['NetIncomeMargin'] = IBC_DCF2['netIncome'] / IBC_DCF2['totalRevenue']
IBC_DCF2['netIncome'].iloc[:4] = IBC_DCF2['totalRevenue'].iloc[:4] * IBC_DCF2[IBC_DCF2['NetIncomeMargin']>0]['NetIncomeMargin'].mean()
IBC_DCF2['FreeCashFlowToEquity'].iloc[:4] = IBC_DCF2['netIncome'].iloc[:4]* IBC_DCF2[IBC_DCF2['FCFE/NetIncome(100%)']>0]['FCFE/NetIncome(100%)'].min()

IBC_DCF2